## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


As the nf-core piplines have strict guidline the data analysis

How would you start: Read through the bioinformatics analysis: dont specify version and parameter how they started runn it. In the paper the method section of this is quite small. So we would need to kind of guess parameter and version and what is needed. 

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

We check on the nf-core website and we would use the rnaseq pipeline.>

To make this run, we first have to prepare our sample sheet so that it looks like this: 
sample,fastq_1,fastq_2,strandedness
CONTROL_REP1,AEG588A1_S1_L002_R1_001.fastq.gz,AEG588A1_S1_L002_R2_001.fastq.gz,auto
CONTROL_REP1,AEG588A1_S1_L003_R1_001.fastq.gz,AEG588A1_S1_L003_R2_001.fastq.gz,auto
CONTROL_REP1,AEG588A1_S1_L004_R1_001.fastq.gz,AEG588A1_S1_L004_R2_001.fastq.gz,auto

The pipline takes the samplesheet and FASTQ files as input, performs quality control (QC), trimming and (pseudo-)alignment, and produces a gene expression matrix and extensive QC report.

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?

Which are the main tools that will be used in the pipeline?

These are the steps from the pipeline: 

    1. Merge re-sequenced FastQ files (cat)
    2. Sub-sample FastQ files and auto-infer strandedness (fq, Salmon)
    3. Read QC (FastQC)
    4. UMI extraction (UMI-tools)
    5. Adapter and quality trimming (Trim Galore!)
    6. Removal of genome contaminants (BBSplit)
    7. Removal of ribosomal RNA (SortMeRNA)
    8. Choice of multiple alignment and quantification routes:
        STAR -> Salmon
        STAR -> RSEM
        HiSAT2 -> NO QUANTIFICATION
    9. Sort and index alignments (SAMtools)
    10. UMI-based deduplication (UMI-tools)
    11. Duplicate read marking (picard MarkDuplicates)
    12. Transcript assembly and quantification (StringTie)
    13. Create bigWig coverage files (BEDTools, bedGraphToBigWig)
    14. Extensive quality control:
            RSeQC
            Qualimap
            dupRadar
            Preseq
            DESeq2
            Kraken2 -> Bracken on unaligned sequences; optional
    15. Pseudoalignment and quantification (Salmon or ‘Kallisto’; optional)
    16. Present QC for raw read, alignment, gene biotype, sample similarity, and strand-specificity checks (MultiQC, R)

The tools are inculded in brackets in the step summary which I copied from the website. 


As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [5]:
# Prepare sample sheet
import pandas as pd
samplesheet = pd.read_csv("fetchngs-out/samplesheet/samplesheet.csv")
samplesheet = samplesheet[["sample", "fastq_1", "fastq_2", "strandedness"]]
samplesheet = samplesheet.replace("SRX19144486", "SRX19144486_SNI_Oxy")
samplesheet = samplesheet.replace("SRX19144488", "SRX19144488_Shm_Oxy")
samplesheet.to_csv("samplesheet_to_run.csv", header=True, index=False)
samplesheet

,sample,fastq_1,fastq_2,strandedness
0,SRX19144486_SNI_Oxy,fetchngs-out/fastq/SRX19144486_SRR23195516_1.f...,fetchngs-out/fastq/SRX19144486_SRR23195516_2.f...,auto
1,SRX19144488_Shm_Oxy,fetchngs-out/fastq/SRX19144488_SRR23195511_1.f...,fetchngs-out/fastq/SRX19144488_SRR23195511_2.f...,auto


In [ ]:
# post here the command you used to run nf-core/rnaseq
!nextflow run nf-core/rnaseq --input samplesheet_to_run.csv --genome GRCm38 --outdir rnaseq -profile docker -c nextflow.config -resume
# use gtv and take the reference genome, but the genome in my thing is correct and we need this reference. 

Nextflow 25.04.7 is available - Please consider updating your version to it

 N E X T F L O W   ~  version 25.04.0

Launching `https://github.com/nf-core/rnaseq` [sleepy_minsky] DSL2 - revision: 9738a2df42 [master]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq 3.21.0
------------------------------------------------------
Input/output options
  input              : samplesheet_to_run.csv
  outdir             : rnaseq

Reference genome options
  genome             : GRCm38
  fasta              : s3://ngi-igenomes/igenomes//Mus_musculus/Ensembl/GRCm38/Sequence/WholeGenomeFasta/genome.fa
  gtf                : s3://ngi-igenomes/igenomes//Mus_musculus/Ensembl/GRCm38/Annotation/Genes/genes.gtf
  gene_bed           : s3://ng

Explain all the parameters you set and why you set them in this way.



## Browsing the results

How did the pipeline perform?

Explain the quality control steps. Are you happy with the quality and why. If not, why not.
Please give additional information on : 
- ribosomal rRNA
- Duplication
- GC content

What are the possible steps that could lead to poorer results?

- ribosomal RNA: It is important to filter ribosomal RNA out of the analysis. rRNA makes around 80% of the RNA in a cell so it might be detected as well. 
    To check this we should look at the featureCounts, 
- Duplication: This means, that some seqeunced are found exactly the same. This coudl be due to a higher expression level, but it could also be due to a PCR overamplificaiton error. In case of the error we would like to exclude it. 
    In the output files they shoudl be a dupRadar. This file plots that relate duplication rate with gene expression levels.
    A good expriment has a higher SI number and show a graph whcih has kind of the shape of a sigmoid curve.
- GC content: This gives a hint about the quality of the reads. A GC bias is an indication, that the library preparation or the PCR went kidn of wrong and introduced some bias. 



Would you exclude any samples? If yes, which and why?

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

The pipeline runs until the DESeq2 analysis. According to the authors, they continued wiht a differential analysis using a 2x2 factorial design. The specify the cutoff for th p-value (<0.05) and for he log2-fold change(<-0.5 or >0.5). Afterwards they performed a VennDiagram generation and a heatmap for reporting in R. Finaly they performed a pathway analysis using IPA.